### Importer les bibliotheques

In [23]:
import pandas  as pd
import requests
import json
import os
from dotenv import load_dotenv
import sqlite3

from datetime import datetime


In [22]:
# Charger le fichier .env
load_dotenv()

True

### on fait un teste pour verfier si le KEY est bien charger ou pas

In [24]:
# Récupérer la clé
api_key = os.getenv("OPENWEATHER_API_KEY")

# Vérifier que la clé est bien chargée
if not api_key:
    raise ValueError("Clé API OpenWeather non trouvée !")

# Exemple d'URL pour Paris
url = f"https://api.openweathermap.org/data/2.5/weather?q=Paris&units=metric&lang=fr&appid={api_key}"

print(url)  # juste pour vérifier que l'URL est correcte

https://api.openweathermap.org/data/2.5/weather?q=Paris&units=metric&lang=fr&appid=9f3aa91a700f53160b04433c00a8e448


In [39]:
# Liste des villes
villes = [
    "Paris", "Lyon", "Marseille", "Toulouse", "Nice",
    "Bordeaux", "Lille", "Nantes", "Strasbourg", "Montpellier",
    "Rennes", "Grenoble", "Dijon", "Avignon", "Reims", "Metz",
]


# Créer une liste pour stocker les résultats
donnees_meteo = []


# Boucle sur chaque ville
for ville in villes:
    url = f"https://api.openweathermap.org/data/2.5/weather?q={ville}&units=metric&lang=fr&appid={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        # Extraire les infos utiles
        ville_data = {
            "ville": ville,
            "temp": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "description": data["weather"][0]["description"],
            "wind_speed": data["wind"]["speed"],
            "date_releve": datetime.now() 
        }
        donnees_meteo.append(ville_data)
    else:
        print(f"Erreur pour la ville {ville}: {response.status_code}")

# Convertir en DataFrame pour nettoyage et manipulation
df_meteo = pd.DataFrame(donnees_meteo)
df_meteo.head()

,ville,temp,humidity,description,wind_speed,date_releve
0,Paris,1.50,87,ciel dégagé,3.60,2025-11-21 20:00:14.902067
1,Lyon,-1.16,84,couvert,3.11,2025-11-21 20:00:15.002066
2,Marseille,2.20,64,couvert,5.81,2025-11-21 20:00:15.114667
3,Toulouse,2.99,81,ciel dégagé,7.20,2025-11-21 20:00:15.217667
4,Nice,-4.33,69,légères chutes de neige,2.66,2025-11-21 20:00:15.323669


### Analyser le dataframe

In [26]:
# voir les types de données
df_meteo.dtypes

ville                  object
temp                  float64
humidity                int64
description            object
wind_speed            float64
date_releve    datetime64[ns]
dtype: object

In [28]:
# Créer ou connecter à la base SQLite
conn = sqlite3.connect("meteo.db")
cursor = conn.cursor()


In [33]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS meteo (
    ville TEXT,
    temp REAL,
    humidity INTEGER,
    description TEXT,
    wind_speed REAL,
    date_releve TEXT
)
""")
conn.commit()


In [34]:
for _, row in df_meteo.iterrows():
    cursor.execute("""
    INSERT INTO meteo (ville, temp, humidity, description, wind_speed, date_releve)
    VALUES (?, ?, ?, ?, ?, ?)
    """, (
        row['ville'],
        row['temp'],
        row['humidity'],
        row['description'],
        row['wind_speed'],
        row['date_releve'].strftime("%Y-%m-%d %H:%M:%S")  # convertir datetime en texte
    ))

conn.commit()


In [35]:
conn.close()
print("Données météo insérées avec succès dans la base SQL !")


Données météo insérées avec succès dans la base SQL !


### Test

In [36]:
import sqlite3
import pandas as pd

# Se connecter à la base SQLite
conn = sqlite3.connect("meteo.db")


In [38]:
# Lire toutes les données de la table "meteo"
df = pd.read_sql_query("SELECT * FROM meteo", conn)

# Afficher les 10 premières lignes
df.head()


,ville,temp,humidity,description,wind_speed,date_releve
0,Paris,1.57,87,ciel dégagé,3.60,2025-11-21 19:42:02
1,Lyon,-1.16,84,couvert,0.89,2025-11-21 19:42:02
2,Marseille,2.28,62,couvert,2.24,2025-11-21 19:42:02
3,Toulouse,2.99,81,nuageux,7.72,2025-11-21 19:42:02
4,Nice,-4.33,69,légères chutes de neige,2.86,2025-11-21 19:42:02
